In [14]:
# 🏁 AUTONOMOUS DRIVING - Using YOUR Working Camera Code
import torch
import torchvision
import numpy as np
import time
import cv2
from utils import preprocess

# Your exact working custom classes
import traitlets
import smbus

class Racecar(traitlets.HasTraits):
    steering = traitlets.Float()
    throttle = traitlets.Float()
    
    @traitlets.validate('steering')
    def _clip_steering(self, proposal):
        return max(-1.0, min(1.0, proposal['value']))
        
    @traitlets.validate('throttle')
    def _clip_throttle(self, proposal):
        return max(-1.0, min(1.0, proposal['value']))

class NvidiaRacecar(Racecar):
    i2c_address = traitlets.Integer(default_value=0x40)
    steering_gain = traitlets.Float(default_value=-0.65)
    steering_offset = traitlets.Float(default_value=0.17)  # YOUR CALIBRATED VALUE!
    steering_channel = traitlets.Integer(default_value=0)
    throttle_gain = traitlets.Float(default_value=0.8)
    throttle_channel = traitlets.Integer(default_value=1)
    
    def __init__(self, *args, **kwargs):
        super(NvidiaRacecar, self).__init__(*args, **kwargs)
        self.bus = smbus.SMBus(7)
        self.center_pulse = 1500
        self._initialize_pca9685()
        print("✅ NvidiaRacecar ready for autonomous driving!")
        print(f"✅ Using calibrated steering offset: {self.steering_offset}")
    
    def _initialize_pca9685(self):
        try:
            self.bus.write_byte_data(self.i2c_address, 0x00, 0x10)
            time.sleep(0.005)
            prescale = int(25000000 / (4096 * 50) - 1)
            self.bus.write_byte_data(self.i2c_address, 0xFE, prescale)
            self.bus.write_byte_data(self.i2c_address, 0x00, 0x20)
            time.sleep(0.005)
            self.bus.write_byte_data(self.i2c_address, 0x01, 0x04)
            self._set_servo_pulse(0, self.center_pulse)
            self._set_servo_pulse(1, self.center_pulse)
        except Exception as e:
            print(f"Warning: {e}")
    
    def _set_servo_pulse(self, channel, pulse_us):
        pulse_us = max(1000, min(2000, pulse_us))
        pwm_value = int((pulse_us * 4096) / 20000)
        base_reg = 0x06 + 4 * channel
        try:
            self.bus.write_byte_data(self.i2c_address, base_reg, 0)
            self.bus.write_byte_data(self.i2c_address, base_reg + 1, 0)
            self.bus.write_byte_data(self.i2c_address, base_reg + 2, pwm_value & 0xFF)
            self.bus.write_byte_data(self.i2c_address, base_reg + 3, (pwm_value >> 8) & 0xFF)
        except Exception as e:
            print(f"PWM Error: {e}")
    
    @traitlets.observe('steering')
    def _on_steering(self, change):
        scaled = change['new'] * self.steering_gain + self.steering_offset
        pulse_us = self.center_pulse + (scaled * 500)
        self._set_servo_pulse(self.steering_channel, pulse_us)
    
    @traitlets.observe('throttle')
    def _on_throttle(self, change):
        scaled = change['new'] * self.throttle_gain
        if scaled > 0:
            pulse_us = self.center_pulse - (scaled * 200)
        elif scaled < 0:
            pulse_us = self.center_pulse + (abs(scaled) * 200)
        else:
            pulse_us = self.center_pulse
        self._set_servo_pulse(self.throttle_channel, pulse_us)

print("🏁 JETRACER AUTONOMOUS DRIVING SYSTEM")
print("Using YOUR working camera code and calibrated car")
print("=" * 50)

# Initialize camera using YOUR working method
def initialize_camera():
    """Initialize camera using your proven working method"""
    print("🎬 Initializing camera using your working method...")
    try:
        from jetcam.csi_camera import CSICamera
        camera = CSICamera(width=224, height=224, capture_fps=21)
        camera.running = True
        print("✅ Camera created and started")
        
        # Wait for camera to warm up (like in your code)
        time.sleep(3)
        
        # Test camera using your method
        test_img = camera.value
        if test_img is not None:
            print(f"✅ Camera working! Shape: {test_img.shape}")
            return camera
        else:
            print("❌ Camera not capturing")
            return None
    except Exception as e:
        print(f"❌ Camera initialization failed: {e}")
        return None

def get_camera_image(camera):
    """Get image from camera using YOUR working method"""
    if camera is None:
        return None
    try:
        img = camera.value  # Use .value for CSICamera when running (your method)
        if img is not None and img.shape[:2] != (224, 224):
            img = cv2.resize(img, (224, 224))
        return img
    except Exception as e:
        print(f"Error getting camera image: {e}")
        return None

def load_trained_model():
    """Load your CPU-trained model"""
    print("🤖 Loading your 371-image trained model...")
    CATEGORIES = ['apex']
    
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
    model = model.float()
    
    try:
        model.load_state_dict(torch.load('road_following_model_cpu.pth', 
                                       map_location='cpu', weights_only=True))
        model.eval()
        print("✅ Your trained model loaded successfully!")
        return model
    except Exception as e:
        print(f"❌ Failed to load model: {e}")
        return None

class DrivingParams:
    THROTTLE = 0.15          # Start VERY slow for safety
    STEERING_GAIN = 0.75     # How responsive AI steering is  
    STEERING_BIAS = 0.0      # Additional AI steering correction
    MAX_STEERING = 1.0       # Safety limits
    MIN_STEERING = -1.0
    UPDATE_RATE = 0.05       # 20Hz

def test_hardware():
    """Test your hardware using your working camera method"""
    print("\n🧪 TESTING HARDWARE")
    print("=" * 30)
    
    print("📷 Testing camera using your working method...")
    camera = initialize_camera()
    
    camera_ok = False
    if camera is not None:
        img = get_camera_image(camera)
        if img is not None:
            print(f"✅ Camera working: {img.shape}")
            camera_ok = True
        else:
            print("❌ Camera not capturing images")
        
        # Stop camera
        camera.running = False
    else:
        print("❌ Camera failed to start")
    
    print("🚗 Testing your calibrated car...")
    car_ok = False
    try:
        car = NvidiaRacecar()  # Uses your 0.17 offset
        print(f"  ✅ Car initialized with steering_offset = {car.steering_offset}")
        
        print("  Testing calibrated steering...")
        car.steering = 0.0   # Should go straight with your calibration
        time.sleep(1)
        car.steering = 0.2   # Right turn
        time.sleep(0.5)
        car.steering = -0.2  # Left turn
        time.sleep(0.5)
        car.steering = 0.0   # Back to straight
        print("  ✅ Calibrated steering test complete")
        
        print("  Testing throttle (gentle)...")
        car.throttle = 1
        time.sleep(0.3)
        car.throttle = 0.0
        print("  ✅ Throttle test complete")
        
        car_ok = True
        
    except Exception as e:
        print(f"❌ Car test failed: {e}")
    
    if camera_ok and car_ok:
        print("\n✅ ALL HARDWARE TESTS PASSED!")
        print("🚀 Your calibrated JetRacer is ready for autonomous driving!")
        return True
    else:
        print("\n❌ Hardware tests failed - fix issues before driving")
        return False

def start_autonomous_driving():
    """Main autonomous driving function using your working camera code"""
    print("\n🏁 STARTING AUTONOMOUS DRIVING!")
    print("=" * 40)
    
    # Load model
    model = load_trained_model()
    if model is None:
        print("❌ Cannot drive without model!")
        return
    
    print("⚠️  SAFETY REMINDERS:")
    print("   • Keep manual override ready!")
    print("   • Clear track with good lighting")
    print("   • Be ready to catch the car")
    print("   • Press Ctrl+C to stop")
    print(f"   • Using your calibrated steering (offset = 0.17)")
    print(f"   • Starting with SLOW throttle: {DrivingParams.THROTTLE}")
    
    print("\n⏱️  Starting in 5 seconds...")
    for i in range(5, 0, -1):
        print(f"   {i}...")
        time.sleep(1)
    
    print("🚀 GO!")
    
    # Initialize hardware using YOUR working methods
    camera = initialize_camera()
    car = NvidiaRacecar()  # Uses your 0.17 offset automatically
    
    if camera is None:
        print("❌ Camera failed to start!")
        return
    
    car.throttle = DrivingParams.THROTTLE
    print(f"🚗 Throttle set to {DrivingParams.THROTTLE}")
    print(f"🎯 Using calibrated steering offset: {car.steering_offset}")
    
    frame_count = 0
    start_time = time.time()
    steering_history = []
    
    try:
        print("🎯 Autonomous driving active with your 371-image model...")
        
        while True:
            # Get camera image using YOUR working method
            image = get_camera_image(camera)
            if image is None:
                print("⚠️ No image from camera")
                continue
            
            # AI prediction
            processed_image = preprocess(image)
            with torch.no_grad():
                output = model(processed_image)
                steering_prediction = float(output[0][0])  # AI's steering prediction
            
            # Apply AI steering parameters (on top of your hardware calibration)
            ai_steering_command = (steering_prediction * DrivingParams.STEERING_GAIN + 
                                 DrivingParams.STEERING_BIAS)
            
            # Safety clamp
            ai_steering_command = max(DrivingParams.MIN_STEERING, 
                                    min(DrivingParams.MAX_STEERING, ai_steering_command))
            
            # Send to your calibrated car (which adds the 0.17 offset automatically)
            car.steering = ai_steering_command
            
            # Monitoring
            frame_count += 1
            steering_history.append(ai_steering_command)
            
            if frame_count % 50 == 0:
                elapsed = time.time() - start_time
                fps = frame_count / elapsed
                avg_steering = np.mean(steering_history[-50:])
                print(f"📊 Frame {frame_count} | FPS: {fps:.1f} | "
                      f"AI Steering: {ai_steering_command:+.3f} | "
                      f"Avg: {avg_steering:+.3f}")
            
            time.sleep(DrivingParams.UPDATE_RATE)
            
    except KeyboardInterrupt:
        print("\n🛑 STOPPING AUTONOMOUS DRIVING")
        
    except Exception as e:
        print(f"\n❌ Error during driving: {e}")
        
    finally:
        # Emergency stop
        try:
            car.throttle = 0.0
            car.steering = 0.0
            print("🛑 Car stopped safely")
        except:
            print("⚠️ Error stopping car - use manual override!")
        
        try:
            if camera is not None:
                camera.running = False
            print("📷 Camera stopped")
        except:
            pass
        
        # Final stats
        elapsed = time.time() - start_time
        avg_fps = frame_count / elapsed if elapsed > 0 else 0
        print(f"📊 Final Stats: {frame_count} frames in {elapsed:.1f}s ({avg_fps:.1f} FPS)")
        
        if steering_history:
            avg_steering = np.mean(steering_history)
            steering_range = max(steering_history) - min(steering_history)
            print(f"📊 AI Steering Stats: Avg={avg_steering:+.3f}, Range={steering_range:.3f}")

# Quick test function
def quick_camera_test():
    """Quick test of your camera method"""
    print("📷 Testing your camera method...")
    camera = initialize_camera()
    if camera:
        img = get_camera_image(camera)
        if img is not None:
            print(f"✅ Camera test passed: {img.shape}")
        camera.running = False
        return True
    return False

print("\n🎮 READY TO DRIVE WITH YOUR PROVEN METHODS!")
print("Functions available:")
print("  quick_camera_test() - Test your camera method")
print("  test_hardware() - Full hardware test")
print("  start_autonomous_driving() - GO RACING!")

print(f"\n⚙️ CURRENT SETTINGS:")
print(f"   Camera: CSICamera(224x224, 21fps) - YOUR WORKING METHOD")
print(f"   Hardware steering offset: 0.17 (your calibration)")
print(f"   AI throttle: {DrivingParams.THROTTLE} (start slow!)")
print(f"   AI steering gain: {DrivingParams.STEERING_GAIN}")

print(f"\n🏁 Your proven camera + calibrated hardware + 371-image model = READY!")

🏁 JETRACER AUTONOMOUS DRIVING SYSTEM
Using YOUR working camera code and calibrated car

🎮 READY TO DRIVE WITH YOUR PROVEN METHODS!
Functions available:
  quick_camera_test() - Test your camera method
  test_hardware() - Full hardware test
  start_autonomous_driving() - GO RACING!

⚙️ CURRENT SETTINGS:
   Camera: CSICamera(224x224, 21fps) - YOUR WORKING METHOD
   Hardware steering offset: 0.17 (your calibration)
   AI throttle: 0.15 (start slow!)
   AI steering gain: 0.75

🏁 Your proven camera + calibrated hardware + 371-image model = READY!


In [12]:
quick_steering_test()

🚗 Quick test of your calibrated system...
✅ NvidiaRacecar ready for autonomous driving!
✅ Using calibrated steering offset: 0.17
Testing straight driving (should go straight)...
✅ Calibrated steering test complete!


In [15]:
test_hardware()


🧪 TESTING HARDWARE
📷 Testing camera using your working method...
🎬 Initializing camera using your working method...
❌ Camera initialization failed: Could not initialize camera.  Please see error trace.
❌ Camera failed to start
🚗 Testing your calibrated car...
✅ NvidiaRacecar ready for autonomous driving!
✅ Using calibrated steering offset: 0.17
  ✅ Car initialized with steering_offset = 0.17
  Testing calibrated steering...


Error generated. /dvs/git/dirty/git-master_linux/multimedia/nvgstreamer/gst-nvarguscamera/gstnvarguscamerasrc.cpp, execute:805 Failed to create CaptureSession
[ WARN:0@321.494] global cap_gstreamer.cpp:1728 open OpenCV | GStreamer warning: Cannot query video position: status=0, value=-1, duration=-1


  ✅ Calibrated steering test complete
  Testing throttle (gentle)...
  ✅ Throttle test complete

❌ Hardware tests failed - fix issues before driving


False